### Use seq2seq model

Demo

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-en-ROMANCE',return_dict=True)

In [3]:
tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-ROMANCE')

In [4]:
content = 'This is a pen'

In [5]:
inputs = tokenizer.encode( '>>es<< ' + content,return_tensors='pt')
outputs = model.generate(inputs,max_length=100,num_beams=4,early_stopping=True)

In [6]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Esto es un bolígrafo


---------------------------------------

In [1]:
import pandas as pd

In [2]:
df = pd.read_json('../augmented_labeled_news.json')

In [3]:
df[:5]

,category,head,content
0,WEIRD NEWS,Animal Shelters edition Of The Bachelor Is Way...,There's fierce competition for the affection o...
1,FOOD & DRINK,Mistakes You're Making With Fried Eggs (PHOTOS),How can something that provides so much also c...
2,SPORTS,Are You Ready For Some L.A. Football?,hoosier state an era when cities are giving hu...
3,WEDDINGS,13 Reminders That This Really Is The Most Wond...,Theyre getting jennjennxo I asked from the top...
4,MISCELLANEOUS,Quick-Thinking 5-Year-Old Saves Blind Grandmot...,She told my mom to hold her shoulder and that ...


In [4]:
en_head = df['head'].tolist()
en_des = df['content'].tolist()

In [5]:
from tqdm import tqdm
from transformers import MarianMTModel,MarianTokenizer
import torch

In [6]:
model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
model.to('cuda')
seqlen = len(en_head)

In [10]:
es_head = []
es_des = []
batch_size = 30
for epoch in tqdm(range(seqlen//batch_size)):
    hsrc = ['>>es<< ' + content for content in en_head[epoch*batch_size:min((epoch+1)*batch_size,seqlen)]]
    dsrc = ['>>es<< ' + content for content in en_des[epoch*batch_size:min((epoch+1)*batch_size,seqlen)]]
    h_input = tokenizer(hsrc,return_tensors="pt",padding=True).to('cuda')
    d_input = tokenizer(dsrc,return_tensors="pt",padding=True).to('cuda')
    h_translated = model.generate(**h_input)
    h_translated.to('cpu')
    h_tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in h_translated]
    
    d_translated = model.generate(**d_input)
    d_translated.to('cpu')
    d_tgt_text = [tokenizer.decode(t, skip_special_tokens=True) for t in d_translated]
    es_head += h_tgt_text
    es_des += d_tgt_text
    torch.cuda.empty_cache()

100%|██████████| 28373/28373 [18:03:37<00:00,  2.29s/it]   


In [11]:
print(len(es_des))
print(len(en_head))
print(len(es_head))
labels = df['category'].tolist()[:-4]
print(len(labels))

851190
851194
851190
851190


In [12]:

es_df = {'category':labels,
     'headline':es_head,
     'description':es_des}
edf = pd.DataFrame(data=es_df)
edf.to_json('./augmented_es_data.json')

In [13]:
esdf = pd.read_json('./augmented_es_data.json')

In [14]:
esdf.head(10)

,category,headline,description
0,WEIRD NEWS,Edición de refugios de animales de la licencia...,Hay una feroz competencia por el afecto de Ste...
1,FOOD & DRINK,Errores que estás haciendo con huevos fritos (...,¿Cómo puede algo que proporciona tanto también...
2,SPORTS,¿Estás listo para un poco de fútbol de Los Áng...,una época en la que las ciudades están dando e...
3,WEDDINGS,13 Recuerdos de que esta es realmente la época...,Ellos están recibiendo Jennjennxo pedí desde e...
4,MISCELLANEOUS,Pensando rápido de 5 años de edad salva a la a...,Le dijo a mi mamá que sostuviera su hombro y q...
5,DIVORCE,Por qué los divorcios de celebridad también pu...,Cuando el ciberespacio me cuenta que Amy Poehl...
6,COMEDY,Una mirada a Bernie más suave Álbum de palabra...,Echa un vistazo a estos cutos profundos de Ber...
7,GROUPS VOICES,"Donald Glover 'Esto es América', a través de l...",De vez en cuando un incidente racial o una exp...
8,FOOD & DRINK,Un jamón de rhubarb de fresa para hacer que la...,Otra manera de celebrar el feliz matrimonio de...
9,WEIRD NEWS,El periodista de TV lo pierde mientras el loro...,"""No puedo quitármelo de encima, oh Dios mío, o..."
